<a href="https://colab.research.google.com/github/TollanBerhanu/Semantic-search-on-Slack/blob/main/Slack_Semantic_Search_(Instructor_Embeddings_%2B_Chroma_DB_%2B_LLaMA2_7B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
import git
import json
import os
import pandas as pd

repo_url = "https://github.com/Yagth/MatterMost-LLM-test-Slack-export-Jun-19-2023---Jun-20-2023.git"
slack_data_path = "/content/slackdata/"
# slack_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset/slack-data/'

git.Repo.clone_from(repo_url, slack_data_path)

<git.repo.base.Repo '/content/slackdata/.git'>

In [ ]:
slack_data_path = "/content/slackdata/"

In [ ]:

def get_all_channels(path):
  df = pd.read_json(path + 'channels.json')

  channel_ids = [id for id in df['id']]
  channel_names = [ name for name in df['name']]

  return pd.DataFrame({ 'channel_id': channel_ids, 'channel_name': channel_names } )

channels = get_all_channels(slack_data_path)
channels

,channel_id,channel_name
0,C05D1SE01B7,random
1,C05D77W3N76,general
2,C05D7863DRA,test
3,C05ABCDE01,gptgenerated


In [ ]:
import glob
import json

# Return the metadata of each message in the channel
def extract_channel_metadata(path, channel_name):
  # use glob to get all the json files in the folder
  daily_json_files = glob.glob(path + channel_name +'/*.json')

  # just return if the channel doesn't exist (or hasn't been exported yet)
  if not daily_json_files:
    return

  metadata = pd.DataFrame(columns = ['message', 'channel', 'date', 'time', 'user_id', 'user_name'])

  # loop over the list of json files (each json file includes every post in that channel for a single day)
  for f in daily_json_files:
    # read the json file
    # today_data = pd.read_json(f)
    with open(f, 'r') as file:
        # Read the contents
        data = file.read()
        # Parse the JSON data
        today_data = json.loads(data)

    today_date = f.split("/")[-1]  # 'f' is the full file path and file name
    print('Extracting...', today_date) # the file name is the date

    # iterate through all the messages of the day
    for msg_data in today_data:
      # Skip if its a "channel_join" type message or if the actual message content is empty
      if ('subtype' in msg_data) or (msg_data['text'] == "") or (msg_data['type'] != 'message'):
        continue
        # TODO: filter out any links, stickers, and other junk
        # TODO: replace @Member references by their real names

      metadata.loc[len(metadata)] = {
            'message': msg_data['text'],
            'channel': channel_name,
            'date': today_date.split(".json")[0], # omit the file extension '.json'
            'time': msg_data['ts'],
            'user_id': msg_data['user'],
            'user_name': msg_data['user_profile']['first_name'] # We can also use 'real_name' if we wanted the full name of the user
      }

  return metadata

## 3. Generating Embeddings

### HF Instructor Embeddings

In [ ]:
!pip -q install langchain chromadb sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 123.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
                                                      # model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer


In [ ]:
# Return a list of embeddings for all messages of the channel
def embed_channel_messages(channel_metadata):
  msg_list = channel_metadata['message'].astype(str).tolist()
  return instructor_embeddings.embed_documents(msg_list)

## Setting up Chroma DB


In [ ]:
import chromadb

In [ ]:
client = chromadb.PersistentClient(path="/chroma/db")

In [ ]:
sample_collection = client.get_or_create_collection(  # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
      name= "slack_collection",
      metadata= {"hnsw:space": "cosine"},
      # embedding_function= instructor_embeddings       # The default embedding model is 'all-MiniLM-L6-V2'
    )

In [ ]:
sample_collection.add(
    documents=["lorem ipsum...", "doc2", "doc3"],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    ids=["id1", "id2", "id3"]
)

In [ ]:
sample_collection.get(
    ids=["id1", "id2", "id3"],
    where={"metadata": "filter"}
)

{'ids': ['id2'],
 'embeddings': None,
 'metadatas': [{'chapter': '3', 'verse': '5'}],
 'documents': ['doc2']}

In [ ]:
# client.delete_collection(name="slack_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

In [ ]:
# This will be the size of the batch of vectors upserted at a time
step = 1000

def upsert_channel_embeddings(channel_name, channel_embeddings, channel_data):

  no_embeddings = len(channel_embeddings)

  parsed_channel_data = json.loads(channel_data.to_json(orient="records"))

  for start in range(0, no_embeddings, step):
    # The end location of the current batch
    end = min(no_embeddings, start+step)    # If it reached the last batch, the end should be the total amount of vectors
                                            # [0..99], [100..199], ... , [1600..1678]  (The last batch should end at 1678)

    # create IDs for all embedded chunks (vectors) ... [channelname_0 -> ... -> channelname_..]
    ids = [channel_name+str(x) for x in range(start, end)]

    # create a records list of current batch for upsert
    # records = zip(ids, channel_embeddings[start:end], parsed_channel_data[start:end]) # "vector_id" , [embeddings], {metadata}

    # upsert to Pinecone
      # vectors = [ ( "id1", [0.1,0.2,..], {metadata1} )  ,  ( "id2", [0.4,0.6,..], {metadata2} )  , .. ]
      # namespace = "my-namespace"
    sample_collection.upsert(
      ids = ids,
      embeddings = channel_embeddings[start:end],
      metadatas = parsed_channel_data[start:end],
      documents = parsed_channel_data['messages'][start:end]
    )

  print(sample_collection.peek()) # returns a list of the first 10 items in the collection
  print(sample_collection.count()) # returns the number of items in the collection

In [ ]:
# Store all messages using the helper functions defined above
def upsert_all():
  channel_names = channels['channel_name'].tolist()

  for channel_name in channel_names:

    channel_metadata = extract_channel_metadata(slack_data_path, channel_name)

    channel_embeddings = embed_channel_messages(channel_metadata)

    upsert_channel_embeddings(channel_name, channel_embeddings, channel_metadata)

In [ ]:
upsert_all()

In [ ]:
# Load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory= '/chroma/db')

## Make a retriever

In [ ]:
def get_context_from_pinecone(query):
  # Generate embeddings for the query
  embedded_query = instructor_embeddings.embed_query(query)

  query_response = sample_collection.query(
      query_embeddings = embedded_query,
      n_results = 10,
      # where = {"metadata_field": "is_equal_to_this"},
      where = {
          "channel": {"$in": ["general", "random"]}
          # "user_id": {"$in": ["U05D1SQDNSH", "U05DHDPL4FK", "U05CQ93C3FZ", "U05D4M7RGQ3"]}
      }
      # where_document={"$contains":"search_string"}
  )

  print(query_response)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("What is Flash attention?")

In [ ]:
len(docs)

4

In [ ]:
docs[0]

Document(page_content='access.\nWe propose FlashAttention , a new attention algorithm that computes exact attention with far fewer\nmemory accesses. Our main goal is to avoid reading and writing the attention matrix to and from HBM.\nThis requires (i) computing the softmax reduction without access to the whole input (ii) not storing the large\nintermediate attention matrix for the backward pass. We apply two well-established techniques to address\nthese challenges. (i) We restructure the attention computation to split the input into blocks and make several\npasses over input blocks, thus incrementally performing the softmax reduction (also known as tiling). (ii) We\nstore the softmax normalization factor from the forward pass to quickly recompute attention on-chip in the\nbackward pass, which is faster than the standard approach of reading the intermediate attention matrix from\nHBM. We implement FlashAttention in CUDA to achieve ﬁne-grained control over memory access and', metadata={'

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
retriever.search_type

'similarity'

In [ ]:
retriever.search_kwargs

{'k': 3}

## Make a chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

ValidationError: ignored

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 FlashAttention is an IO-aware exact attention algorithm that uses tiling to reduce the number of memory
reads/writes between GPU high bandwidth memory (HBM) and GPU on-chip SRAM. It is designed to improve the
quality of Transformers and enable them to scale to longer sequences.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
# break it down
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

 IO-aware means optimizing for reading and writing to fast/slow memory. It has a long history in computer
science and been known by many names, such as the working set model, data locality, the Rooﬂine model of
arithmetic intensity, analyses of scalability, and standard textbook treatments of computer architecture.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Tiling in flash-attention is a technique used to split the input into blocks and make several passes over
input blocks to incrementally perform the softmax reduction.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer is a language model that learns in a self-supervised way how to use different tools such as search
engines, calculators, and translation systems via simple API calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer can use search engines, calculators, and translation systems via simple API calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 We only need a handful of manually labeled examples for few-shot prompting.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 According to the text, dense and sparse retrievers can be used to augment a LM. Dense retrievers work with a
dense query and dense LMs while having less parameters, while sparse retrievers work with sparse bag-of-words
representations of the documents and the queries. Additionally, LMs can be augmented with reasoning strategies
and external tools to provide more relevant context.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What are the differences between REALM and RAG?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 REALM (Guu et al. ,2020) is the first method to jointly train end-to-end a retrieval system with an encoder
LM, while RAG (Lewis et al. ,2020) jointly fine-tunes the retriever with a sequence-to-sequence model.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/ReACT.pdf


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f0cbf5433d0>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [ ]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 5%)
  adding: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 39%)
  adding: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin (deflated 17%)
  adding: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 35%)
  adding: db/chroma-embeddings.parquet (deflated 29%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-collections.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin  
  inflating: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


### Chat prompts

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
